In [1]:
"""
## source : Facebook

## cinenatique
- selection d'un place_id
- recherche des places autour -> liste des places
- recherches des events associés aux places -> liste des events 
- recherches des participants associés aux events -> liste des participants

## resultats :
- fichier liste des places : [facebook_places_dict]
- fichier liste des events : [facebook_events_places_dict]
- fichier liste des participants : [facebook_events_attending_dict]


## TODO
        [1] ajouter le place id utilisé en recherche dans le fichier des events
        [1] gérer le upcoming dans la recherche d'events
"""

In [2]:
import urllib3
#import facebook
import requests
import certifi
import json
import time
import datetime
from random import randrange, uniform

app_id = "000"
app_secret = "000"
access_token = app_id + "|" + app_secret
page_id = 'IALeMans'

In [3]:
def unicode_decode(text):
    try:
        return text.encode('utf-8').decode()
    except UnicodeDecodeError:
        return text.encode('utf-8')

In [4]:
def save_dict(data, name ):
    ts = str(time.time())
    with open('obj/' + name + '_' + ts + '.json', 'w') as fp:
        json.dump(data, fp, sort_keys=True, indent=4)

In [5]:
def request_until_succeed(url):

    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    r = http.request('GET', url)
    success = False
    while success is False:
        try: 
            if r.status == 200:
                success = True
        except Exception as e:
            print (e)
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return r.data

In [6]:
def getFacebookNextPage(data, access_token):

    cursor = 0
    #print(data)
    
    if ('paging') in data:
       #print(data['paging'])
        if ('next') in data['paging']:
            cursor = data['paging']['next']
    
    #print (cursor)
    return cursor

In [7]:
def getFacebookPlaceIdLocation(place_id, access_token):
    
    # construct the URL string
    #base = "https://graph.facebook.com/v2.10"
    base = "https://graph.facebook.com/v2.11"
    node = "/" + place_id
    parameters = "/?fields=location&access_token=%s" % access_token
    url = base + node + parameters

    # retrieve data
    #print (url)
    data = json.loads(request_until_succeed(url))
    
    if ('location') in data:
        latitude = data['location']['latitude']
        longitude = data['location']['longitude']
    else:
        latitude = 0
        longitude = 0
    
    return latitude, longitude

In [8]:
def getFacebookPlacesData(latitude, longitude, distance, cursor, access_token):
    
    if cursor in (0, 1):
        # construct the URL string
        #base = "https://graph.facebook.com/v2.10"
        base = "https://graph.facebook.com/v2.11"
        node = ""
        parameters = "/search?type=place&q=*&center=%s,%s&distance=%s&access_token=%s" % (latitude, longitude, distance, access_token)
        url = base + node + parameters
    else:
        url = cursor

    # retrieve data
    print (url)
    time.sleep(uniform(0.1, 0.8))
    data = json.loads(request_until_succeed(url))
    
    return data

In [9]:
#?fields=location,events{attending_count,end_time,name,attending{name,id}}

def getFacebookEventDataByPlace(page_id, upcoming, cursor, access_token):

    if cursor in (0, 1):
        #base = "https://graph.facebook.com/v2.10"
        base = "https://graph.facebook.com/v2.11"
        node = "/" + page_id + "/events"
        parameters = "/?fields=attending_count,description,end_time,name,place"
        if (upcoming):
            upcoming_string = "&time_filter=upcoming"
        else:
            upcoming_string = ""
        token = "&access_token=%s" % access_token
        url = base + node + parameters + upcoming_string + token
    else:
        url = cursor
 
    # retrieve data
    time.sleep(uniform(0.1, 0.8))
    print (url)
    data = json.loads(request_until_succeed(url))
    
    return data

In [10]:
def getFacebookEventDataAttending(event_id, cursor, access_token):
    
    if cursor in (0, 1):
        base = "https://graph.facebook.com/v2.11"
        node = "/" + event_id 
        parameters = "/attending?access_token=%s" % access_token
        url = base + node + parameters
    else:
        url = cursor
 
    # retrieve data
    time.sleep(uniform(0.1, 0.8))
    print (url)
    data = json.loads(request_until_succeed(url))
    
    return data

In [11]:
def storeFacebookDataList(datas, FacebookDataList , meta):
    
    #print(meta)
    if ('data') in datas:
        #print(datas)
        #print(datas['data'])
        FacebookDataList += datas['data']
    FacebookDataList += meta
            
    return FacebookDataList

### starting point

In [39]:
### recherche coordonnée d'un lieu de référence
# /search?type=place&q=brique24

### init
#center_place_id = "612215925547601" #brique24

### mettre upcoming à True pour le parc expo
#center_place_id = "360719668441" #parc expo le mans

#center_place_id = "252485845229163" #Le Mans Innovation
#center_place_id = "207443273095661" #centre ville le mans
#center_place_id = "118301834894337" #gare du mans

distance = 50
cursor = 1
dict_places = {}
dict_places_data = []


### start exec
latitude, longitude = getFacebookPlaceIdLocation(center_place_id, access_token)

if (latitude == 0):
    print('WARN - not found')
else:
    print ('latitude, longitude ', latitude, longitude)

### recherche lieux dans perimetre (m) autour du lieu de référence
while cursor != 0:
    data_places = getFacebookPlacesData(latitude, longitude, distance, cursor, access_token)
    #print ('cursor ', cursor)
    dict_places_data = storeFacebookDataList(data_places, dict_places_data, "")
    cursor = getFacebookNextPage(data_places, access_token)

#print('places nb ', len(dict_places))

latitude, longitude  47.995630295019 0.19273632635854
https://graph.facebook.com/v2.11/search?type=place&q=*&center=47.995630295019,0.19273632635854&distance=50&access_token=1348018245320570|179af3fa9754e25000b830b189f78304


In [41]:
dict_places_meta = {'timestamp' : time.time(), 'source' : 'facebook', 'result' : 'places', 'request' : 'center distance', 'latitude' : latitude, 'longitude' : longitude, 'distance' : distance}
dict_places = {'meta' : dict_places_meta, 'data' : dict_places_data}
save_dict(dict_places, "facebook_places_dict")    

In [45]:
dict_places_data = []
dict_places_data += [{"name": "Le Mans Université", "id": "943566482390937"}]
dict_places_data += [{"name": "Université du Maine", "id": "109321005753620"}]
dict_places_data += [{"name": "McDonald's Le Mans Université", "id": "1458015740877800"}]
dict_places_data += [{"name": "Université", "id": "110278242899945"}]
dict_places_data += [{"name": "Université-Ribay", "id": "252377111589209"}]
dict_places_data += [{"name": "CIC Banque CIO-BRO", "id": "1115687441876284"}]
dict_places_data += [{"name": "Subway", "id": "108884802526581"}]
dict_places_data += [{"name": "Premiere Classe le Mans Ouest - Universite", "id": "308784242872866"}]
dict_places_data += [{"name": "La Trinquette Université", "id": "785878261543546"}]
dict_places_data += [{"name": "Avum Loisirs Association des Quartiers Voltaire Université Mansart", "id": "143334175710226"}]
dict_places_data += [{"name": "Université du Maine", "id": "252924951765260"}]
dict_places_data += [{"name": "Bsm le Mans", "id": "1668853240084593"}]
dict_places_data += [{"name": "Carrefour Market - Université Du Mans", "id": "188667377845520"}]
dict_places_data += [{"name": "Cabinet infirmier Université", "id": "414176352292331"}]
dict_places_data += [{"name": "Le QG Université du Maine", "id": "140319696064276"}]
dict_places_data += [{"name": "Centre de Santé médico-psycho-social de Le Mans Université", "id": "507240942667351"}]
dict_places_data += [{"name": "Service Culture - Le Mans Université", "id": "164504647050253"}]
dict_places_data += [{"name": "Les Assos du Campus - Le Mans Université", "id": "512861298874868"}]
dict_places_data += [{"name": "Rugby Club du Mans", "id": "150376618369045"}]
dict_places_data += [{"name": "Le Brooklyn  resto", "id": "171097412988789"}]
dict_places_data += [{"name": "Salle Uto'pitre", "id": "149315591756431"}]
dict_places_data += [{"name": "Choeur de l'Université du Mans", "id": "105422056156671"}]
dict_places_data += [{"name": "Pharmacie de l'Université", "id": "277077999384376"}]
dict_places_data += [{"name": "Master 2 Droit Bancaire et Financier - Le Mans Université", "id": "351768851883630"}]
dict_places_data += [{"name": "École nationale supérieure d'ingénieurs du Mans", "id": "162370537116187"}]
dict_places_data += [{"name": "Technopôle de l'Université", "id": "728721720636058"}]
dict_places_data += [{"name": "Licence Acoustique - Le Mans", "id": "487335228136697"}]
dict_places_data += [{"name": "Centre de Ressources en Langues - Université du Maine", "id": "844981208892964"}]
dict_places_data += [{"name": "Master AAF Le Mans", "id": "337220923106607"}]
dict_places_data += [{"name": "DUFOS Université du Maine", "id": "260540853991648"}]
dict_places_data += [{"name": "Master Toxicologie, écotoxicologie Le Mans", "id": "459146804151675"}]
dict_places_data += [{"name": "ATUM - Atelier Théâtre de l'Université du Mans", "id": "333918443443904"}]
dict_places_data += [{"name": "Citavox", "id": "1644992249090657"}]
dict_places_data += [{"name": "Club d'astronomie de l'Université du Maine", "id": "204130229697717"}]
dict_places_data += [{"name": "Cabinet Lecoeur Leduc et Associés", "id": "1289568621129234"}]
dict_places_data += [{"name": "Gefiro", "id": "276286396160314"}]
dict_places_data += [{"name": "Select T T", "id": "413267802365980"}]
dict_places_data += [{"name": "Festiv'Mans", "id": "874298605976631"}]
dict_places_data += [{"name": "Skatepark Le Spot", "id": "196717770663557"}]
dict_places_data += [{"name": "EVE - Scène universitaire", "id": "1843046229314962"}]
dict_places_data += [{"name": "Pressing Bartholdi", "id": "1655505061388982"}]
dict_places_data += [{"name": "O'Potager", "id": "1752493611723460"}]
dict_places_data += [{"name": "Asso Géographes du Maine", "id": "447376618638802"}]
dict_places_data += [{"name": "Charichild's", "id": "146813136064598"}]
dict_places_data += [{"name": "Chaire Unesco \"Pratiques de la philosophie avec les enfants\"", "id": "1755631187993211"}]
dict_places_data += [{"name": "MUNUMANS", "id": "1701755386708312"}]
dict_places_data += [{"name": "BUREAUX & SERVICES", "id": "406799329333998"}]
print(dict_places_data)

[{'name': 'Le Mans Université', 'id': '943566482390937'}, {'name': 'Université du Maine', 'id': '109321005753620'}, {'name': "McDonald's Le Mans Université", 'id': '1458015740877800'}, {'name': 'Université', 'id': '110278242899945'}, {'name': 'Université-Ribay', 'id': '252377111589209'}, {'name': 'CIC Banque CIO-BRO', 'id': '1115687441876284'}, {'name': 'Subway', 'id': '108884802526581'}, {'name': 'Premiere Classe le Mans Ouest - Universite', 'id': '308784242872866'}, {'name': 'La Trinquette Université', 'id': '785878261543546'}, {'name': 'Avum Loisirs Association des Quartiers Voltaire Université Mansart', 'id': '143334175710226'}, {'name': 'Université du Maine', 'id': '252924951765260'}, {'name': 'Bsm le Mans', 'id': '1668853240084593'}, {'name': 'Carrefour Market - Université Du Mans', 'id': '188667377845520'}, {'name': 'Cabinet infirmier Université', 'id': '414176352292331'}, {'name': 'Le QG Université du Maine', 'id': '140319696064276'}, {'name': 'Centre de Santé médico-psycho-soc

[{'id': '118301834894337', 'name': 'Gare du Mans'},
 {'id': '187071001354968', 'name': 'Circuit des 24 Heures'},
 {'id': '812212002138834', 'name': 'Alerte Apéro Le Mans'},
 {'id': '612215925547601', 'name': 'Brique24'},
 {'id': '1866456216910710', 'name': 'BLEU PIXEL'},
 {'id': '1728626470759451', 'name': 'BTP MAP'}]

In [46]:
dict_events_places = []

#print(dict_places_data[0]['name'])
#print(dict_places_data[0]['id'])
#
#for iter in dict_places_data:
#    print(iter['id'])

for value in dict_places_data:
    cursor = 1
    #print(value['id'], True, cursor, access_token)
    while cursor != 0:
        if ('id') in value:
            data_events_places = getFacebookEventDataByPlace(value['id'], False, cursor, access_token)
            dict_events_places = storeFacebookDataList(data_events_places, dict_events_places , [{'events_place' : value['id']}])
            cursor = getFacebookNextPage(data_events_places, access_token)
            print ('cursor ', cursor)
        else:
            cursor = 0

https://graph.facebook.com/v2.11/943566482390937/events/?fields=attending_count,description,end_time,name,place&access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  https://graph.facebook.com/v2.11/943566482390937/events?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&fields=attending_count%2Cdescription%2Cend_time%2Cname%2Cplace&limit=25&after=QVFIUnVnRHUyQXlZAZAlhJZAE8tSVZACUlA1QzEzaWtUc2V2Nzlzek5DeVpCUXZAxcklFSWFVVnVwOUhDdndjRTEwQzlDYWtrMVBXRG1icDdWTkR4NG1kN3dBUzdB
https://graph.facebook.com/v2.11/943566482390937/events?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&fields=attending_count%2Cdescription%2Cend_time%2Cname%2Cplace&limit=25&after=QVFIUnVnRHUyQXlZAZAlhJZAE8tSVZACUlA1QzEzaWtUc2V2Nzlzek5DeVpCUXZAxcklFSWFVVnVwOUhDdndjRTEwQzlDYWtrMVBXRG1icDdWTkR4NG1kN3dBUzdB
cursor  0
https://graph.facebook.com/v2.11/109321005753620/events/?fields=attending_count,description,end_time,name,place&access_token=1348018245320570|179af3fa9754e25

cursor  0
https://graph.facebook.com/v2.11/162370537116187/events/?fields=attending_count,description,end_time,name,place&access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
https://graph.facebook.com/v2.11/728721720636058/events/?fields=attending_count,description,end_time,name,place&access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
https://graph.facebook.com/v2.11/487335228136697/events/?fields=attending_count,description,end_time,name,place&access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
https://graph.facebook.com/v2.11/844981208892964/events/?fields=attending_count,description,end_time,name,place&access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
https://graph.facebook.com/v2.11/337220923106607/events/?fields=attending_count,description,end_time,name,place&access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
https://graph.facebook.com/v2.11/260540853991648/events/?fields=a

In [47]:
#dict_events_places = [{'id':'341679926302892'},{'id':'693485667512953'}, {'id':'115655525776401'}]

#print(dict_events_places)
#for value in dict_events_places:
#    cursor = 1
#    print(value)
    

In [48]:
dict_events_meta = {'timestamp' : time.time(), 'source' : 'facebook', 'result' : 'events', 'request' : 'place'}
dict_events = {'meta' : dict_events_meta, 'data' : dict_events_places}
save_dict(dict_events, "facebook_events_places_dict")  

In [ ]:
dict_events_attending = []

for value in dict_events_places:
    cursor = 1
    if ('id') in value:
        print("id " + value['id'])
        while cursor != 0:
            data_events_attending = getFacebookEventDataAttending(value['id'], cursor, access_token)
            dict_events_attending = storeFacebookDataList(data_events_attending, dict_events_attending, [{'event_id' : value['id']}])
            cursor = getFacebookNextPage(data_events_attending, access_token)
            print ('cursor ', cursor)
    else:
        print("events_place " + value['events_place'])


id 693485667512953
https://graph.facebook.com/v2.11/693485667512953/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 115655525776401
https://graph.facebook.com/v2.11/115655525776401/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  https://graph.facebook.com/v2.11/115655525776401/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUmo3cWJ4S1VoZAEstTFk4UGh1VC1OdjZAWTVhVc1BHSDNmdDY4bld1QThLTURneUk0eU1lV2lpeFpHNFFDb2JrOGQZD
https://graph.facebook.com/v2.11/115655525776401/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUmo3cWJ4S1VoZAEstTFk4UGh1VC1OdjZAWTVhVc1BHSDNmdDY4bld1QThLTURneUk0eU1lV2lpeFpHNFFDb2JrOGQZD
cursor  0
id 295867230882147
https://graph.facebook.com/v2.11/295867230882147/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 543146849355844
https://graph.facebook.com/v2.11/543146849355844/a

cursor  0
id 1444677448916477
https://graph.facebook.com/v2.11/1444677448916477/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 948135781996844
https://graph.facebook.com/v2.11/948135781996844/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 310991335984005
https://graph.facebook.com/v2.11/310991335984005/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 191467481335097
https://graph.facebook.com/v2.11/191467481335097/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 229444717494058
https://graph.facebook.com/v2.11/229444717494058/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  https://graph.facebook.com/v2.11/229444717494058/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUkI5TWFxN3ViUlhvT0hKV1RuZAWVHTDlxOVpMRWlkWVJCS3BaZAHpUVTk0eWNJWFM3Tkd1UmhKRl9qS20zS3JSSEI5YT

https://graph.facebook.com/v2.11/246698479049958/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUlNESnJsVldGMmZAKVWN3TW1VWlVrSWkzdDdJZA2R5cHZAyUUtzazdRcDdnZAE1CWm0yaEZAhZAmhJTmFtYnB6eEtlRHgZD
cursor  https://graph.facebook.com/v2.11/246698479049958/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUkFWUnVwMThQbG5NTHN3ek5lbUJiMmVuN05QOGNQb0ZAhOXYyUjcxVl9Lem0zYjZAwSkhNRHdKd1hjN2RrUUFXZAG0ZD
https://graph.facebook.com/v2.11/246698479049958/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUkFWUnVwMThQbG5NTHN3ek5lbUJiMmVuN05QOGNQb0ZAhOXYyUjcxVl9Lem0zYjZAwSkhNRHdKd1hjN2RrUUFXZAG0ZD
cursor  0
id 320237478315080
https://graph.facebook.com/v2.11/320237478315080/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1004738049594593
https://graph.facebook.com/v2.11/1004738049594593/attending?access_token=1348018245320570|179af3fa97

cursor  0
id 152891054889091
https://graph.facebook.com/v2.11/152891054889091/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 522247587818660
https://graph.facebook.com/v2.11/522247587818660/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 498943443499362
https://graph.facebook.com/v2.11/498943443499362/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 313504918778772
https://graph.facebook.com/v2.11/313504918778772/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
events_place 507240942667351
id 373975303040783
https://graph.facebook.com/v2.11/373975303040783/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 129413257774313
https://graph.facebook.com/v2.11/129413257774313/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 339539403184793
https://graph.facebook.com/v2.11/339

cursor  0
id 1807704029485632
https://graph.facebook.com/v2.11/1807704029485632/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 146934285809013
https://graph.facebook.com/v2.11/146934285809013/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1761211037537399
https://graph.facebook.com/v2.11/1761211037537399/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1227931743956348
https://graph.facebook.com/v2.11/1227931743956348/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 394003327607848
https://graph.facebook.com/v2.11/394003327607848/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  https://graph.facebook.com/v2.11/394003327607848/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUjRKQ3V2OEpMOERMWVppOVZAaOUd3clhSbU13djRrZAG5GclBmemt0clZAMTS05SUw1VFl1cW80ZA2t6Mzh3NFJn

...

https://graph.facebook.com/v2.11/688540454638209/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUmNXVkM4OHF6ZAHRUSy1iS2FFRDJuZA1FLM1lwQU5udWdXUkxpclcydmRVMXJfeTVWenZAsakJYaTBENC04dVVIbmQZD
cursor  0
id 168836326853698
https://graph.facebook.com/v2.11/168836326853698/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  https://graph.facebook.com/v2.11/168836326853698/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUmxPT3hhX0xzX3FjQW9JdzdJOW5YdVlCU2psR1RfUVZAtb1RQY09pRkpVcDU2RkFSbmZABdUNpY1pKVlpreEthQXcZD
https://graph.facebook.com/v2.11/168836326853698/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUmxPT3hhX0xzX3FjQW9JdzdJOW5YdVlCU2psR1RfUVZAtb1RQY09pRkpVcDU2RkFSbmZABdUNpY1pKVlpreEthQXcZD
cursor  https://graph.facebook.com/v2.11/168836326853698/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&lim

https://graph.facebook.com/v2.11/1057503770952111/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUllWdG8wLVNpYzFYYXk4UFRzXzA5SGR1ZAkxzc3pLOVdFcmxkMTVnTHdrcVlmUTZAaZAkQ2UFltVzEwRmE0S284T2UZD
cursor  0
id 1219700384752557
https://graph.facebook.com/v2.11/1219700384752557/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1070703503017397
https://graph.facebook.com/v2.11/1070703503017397/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 131858817299039
https://graph.facebook.com/v2.11/131858817299039/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 448788875291395
https://graph.facebook.com/v2.11/448788875291395/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1677474562565209
https://graph.facebook.com/v2.11/1677474562565209/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cu

https://graph.facebook.com/v2.11/895509377177394/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 992516307449321
https://graph.facebook.com/v2.11/992516307449321/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 809652199111026
https://graph.facebook.com/v2.11/809652199111026/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 948342198517988
https://graph.facebook.com/v2.11/948342198517988/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1013501368669859
https://graph.facebook.com/v2.11/1013501368669859/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 641300295969924
https://graph.facebook.com/v2.11/641300295969924/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 855711701164490
https://graph.facebook.com/v2.11/855711701164490/attending?access_token=1348018245320570|179

cursor  0
id 930808810286353
https://graph.facebook.com/v2.11/930808810286353/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1552392091693414
https://graph.facebook.com/v2.11/1552392091693414/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 733172813386503
https://graph.facebook.com/v2.11/733172813386503/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
events_place 333918443443904
events_place 1644992249090657
id 120171471988086
https://graph.facebook.com/v2.11/120171471988086/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1980114045555965
https://graph.facebook.com/v2.11/1980114045555965/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 279610755473258
https://graph.facebook.com/v2.11/279610755473258/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 277395052378921
ht

cursor  https://graph.facebook.com/v2.11/1706962632941288/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUl9LdF9HR0JwTEs5RjN4SmNmRDNDNVVTT1VRcFozdmZA5NXA5ZAmpWWjh2VUFCeHJGajZA5VEk5MENDNlFyQWdsdG4ZD
https://graph.facebook.com/v2.11/1706962632941288/attending?access_token=1348018245320570%7C179af3fa9754e25000b830b189f78304&limit=25&after=QVFIUl9LdF9HR0JwTEs5RjN4SmNmRDNDNVVTT1VRcFozdmZA5NXA5ZAmpWWjh2VUFCeHJGajZA5VEk5MENDNlFyQWdsdG4ZD
cursor  0
events_place 1843046229314962
events_place 1655505061388982
events_place 1752493611723460
id 1436726866350899
https://graph.facebook.com/v2.11/1436726866350899/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 1786463281612237
https://graph.facebook.com/v2.11/1786463281612237/attending?access_token=1348018245320570|179af3fa9754e25000b830b189f78304
cursor  0
id 632336723579022
https://graph.facebook.com/v2.11/632336723579022/attending?access_token=1348018245320570|179a

In [37]:
print(dict_events_attending)

[{'event_id': '444868235908010'}, {'name': 'Aziza Za', 'id': '2015426932021707', 'rsvp_status': 'attending'}, {'name': 'Anne Bourglan', 'id': '10214820089595744', 'rsvp_status': 'attending'}, {'name': 'Fish McGilvray', 'id': '10159506613390322', 'rsvp_status': 'attending'}, {'name': 'Maité Ritoit', 'id': '915589688589360', 'rsvp_status': 'attending'}, {'name': 'Fanny Nénette', 'id': '10214961389890701', 'rsvp_status': 'attending'}, {'name': 'Wendy Marcais', 'id': '10214790790224374', 'rsvp_status': 'attending'}, {'name': 'Màdmoiselle Yùna', 'id': '10212022146562121', 'rsvp_status': 'attending'}, {'name': 'Romain Camus', 'id': '10155890998918750', 'rsvp_status': 'attending'}, {'event_id': '820590801354302'}, {'name': 'Alicia Lebouleux', 'id': '1502434099839703', 'rsvp_status': 'attending'}, {'name': 'Anthony Chanteloup', 'id': '1478877228847140', 'rsvp_status': 'attending'}, {'name': 'Flavien Simon', 'id': '1929500300400420', 'rsvp_status': 'attending'}, {'name': 'Guillaume Rocher', 'id

In [38]:
dict_events_attending_meta = {'timestamp' : time.time(), 'source' : 'facebook', 'result' : 'people attending events', 'request' : 'events'}
dict_events_att = {'meta' : dict_events_attending_meta, 'data' : dict_events_attending}
save_dict(dict_events_att, "facebook_events_attending")  